In [1]:
import pandas as pd
import numpy  as np
import matplotlib as plt
import seaborn as sns
from IPython.display import HTML, display

# First dataset

In [2]:
epidemiology_df = pd.read_excel('situacion-epidemiologica-coronavirus-en-castilla-y-leon.xlsx').drop(['codigo_ine', 'Posicion'], axis=1).sort_values(by='fecha').reset_index(drop=True)

Problems found with the first dataset ('situacion-epidemiologica-coronavirus-en-castilla-y-leon'):
- Individual problems: 
    + A. Poor assigned names to the different variables that leads to confussion (and no descripcions in the source) -> SOL: Analysis of the data to determine its true meaning
    + B. For some variables the daily and accumulated values are registered, but not for others -> SOL: Preprocessing of the data to generate both values for each variable
- Problems for the merging with the other dataset:
    + C. Different values in the province variable (this dataset has the 'Total_CyL' value missing) -> SOL: Preprocessing of the data to generate the 'Total_CyL' values that summarize the information from the whole autonomous community.

### Solving problem A
##### Assign proper names to the variables

In [3]:
# Change the names and translate
epidemiology_df.columns = ['date', 'province', 'total_positives', 'new_positives', 'total_discharges', 'total_deceases']

### Solving problem B
##### Obtain the daily values of each variable from the accumulated values

In [4]:
# Create the new variables for daily data
epidemiology_df['new_discharges'] = 0
epidemiology_df['new_deceases']   = 0

# Obtain the daily data by substracting the accumulated values from consecutive days
for current_province in epidemiology_df['province'].unique():
    
    rows_current_province = epidemiology_df['province'] == current_province
    data_current_province = epidemiology_df[rows_current_province]
    
    epidemiology_df.loc[rows_current_province,'new_discharges'] = data_current_province['total_discharges'].diff()
    epidemiology_df.loc[rows_current_province,'new_deceases']   = data_current_province['total_deceases'].diff()

# Fix the values for the first day (it has no previous day to substract in the previous step)
epidemiology_df.loc[0:9,'new_discharges'] = epidemiology_df[0:9]['total_discharges']
epidemiology_df.loc[0:9,'new_deceases']   = epidemiology_df[0:9]['total_deceases']

### Solving problem C
##### Generate the 'TotalCyL' value for the 'province' variable by adding the daily information of each of the provinces

In [5]:
# Group the data by date and add the data of all the provinces
epidemiology_CyL_df = epidemiology_df.groupby(by='date').sum()

# Fix the index to keep the date and add the 'province' value
epidemiology_CyL_df = epidemiology_CyL_df.reset_index()
epidemiology_CyL_df['province'] = 'TotalCyL'

# Concatenate the new rows to the previous dataset
epidemiology_df = pd.concat([epidemiology_df, epidemiology_CyL_df]).sort_values(by='date').reset_index(drop=True)

### Final state of the dataset

In [6]:
epidemiology_df.head(10)

,date,province,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases
0,2020-03-13,Burgos,90.0,29.0,0.0,1.0,0.0,1.0
1,2020-03-13,Valladolid,11.0,7.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-13,León,32.0,22.0,0.0,0.0,0.0,0.0
4,2020-03-13,Ávila,3.0,3.0,0.0,0.0,0.0,0.0
5,2020-03-13,TotalCyL,169.0,77.0,1.0,3.0,1.0,3.0
6,2020-03-13,Segovia,15.0,6.0,0.0,1.0,0.0,1.0
7,2020-03-13,Zamora,5.0,3.0,0.0,0.0,0.0,0.0
8,2020-03-13,Salamanca,11.0,5.0,1.0,1.0,1.0,1.0
9,2020-03-13,Soria,1.0,1.0,0.0,0.0,0.0,0.0


# Second dataset

In [7]:
vacunation_df = pd.read_excel('personas-vacunadas-covid.xlsx').sort_values(by='fecha').reset_index(drop=True)

Problems found with the first dataset ('personas-vacunadas-covid'):
- Individual problems: 
    + A. Some variables have been registered since the creation of the dataset and others have been included later, derivating in confusing names and numerous missing values -> SOL: Analysis of the data to determine its true meaning and the cause of the missings
    + B. For some variables the absolute and percentage values are registered, but not for others -> SOL: Preprocessing of the data to generate both values for each variable
- Problems for the merging with the other dataset:
    + C. Different values in the province variable (this dataset has the 'Total_CyL' value ) -> SOL: Solved in the previous dataset

### Solving problem A
##### 1. Assign proper names to the variables

In [8]:
# Change the names and translate
vacunation_df.columns = ['date', 'province', 'new_doses_administered', 'new_persons_two_doses', 'new_persons_one_dose', 'percentage_persons_one_dose', 'percentage_persons_two_doses',  'new_persons_three_doses', 'percentage_persons_three_doses']

##### 2. Determine the meaning of the missings and set values if needed

In this dataset we have 2 different cases:
- 'Percentage_persons_one_dose'and 'percentage_persons_two_doses' were not measured since the begining but the data can be obtained from the original variables.
- 'New_persons_three_doses' and 'percentage_persons_three_doses' weren't measured since the begining either, but their data is zero until the first value in 'New_persons_three_doses', since third doses had not been given yet.

To fill the missings from the 'percentage' columns case we will obtain the population of each province and use it after solving problem B (because we need data that will be generated there).

However, to fill the 'New_persons_three_doses', we will just input zeroes in the missing values.

In [10]:
population_province = {}

for current_province in epidemiology_df['province'].unique():
    
    total_number_persons_one_dose     = vacunation_df[vacunation_df['province']==current_province]['new_persons_one_dose'].sum()
    final_percentage_persons_one_dose = vacunation_df[vacunation_df['province']==current_province]['percentage_persons_one_dose'].max()

    population_province[current_province] = total_number_persons_one_dose / final_percentage_persons_one_dose * 100
    
population_province

{'Burgos': 347828.75779106736,
 'Valladolid': 509082.53635826905,
 'Palencia': 157223.49458812363,
 'León': 447229.52868307946,
 'Ávila': 156404.79154812612,
 'TotalCyL': 2391847.508462069,
 'Segovia': 150285.6904653503,
 'Zamora': 166534.37871694405,
 'Salamanca': 318453.8270868207,
 'Soria': 87513.51814894995}

In [11]:
vacunation_df['new_persons_three_doses'] = vacunation_df['new_persons_three_doses'].fillna(0)

### Solving problem B
##### Obtain the accumulated values of each variable from the daily new values
In this step we will also fill the missings from the 'percentage' columns mentioned in problem A, solving the remaining problem.

In [16]:
# Create the new variables for daily data
vacunation_df['total_doses_administered'] = 0

vacunation_df['total_number_persons_one_dose'] = 0
vacunation_df['total_number_persons_two_doses'] = 0
vacunation_df['total_number_persons_three_doses'] = 0

# Obtain the accumulated data by adding the values from consecutive days
for current_province in vacunation_df['province'].unique():
    
    rows_current_province = vacunation_df['province'] == current_province
    data_current_province = vacunation_df[rows_current_province]
    
    vacunation_df.loc[rows_current_province,'total_doses_administered'] = data_current_province['new_doses_administered'].cumsum()
    
    vacunation_df.loc[rows_current_province,'total_number_persons_one_dose'] = data_current_province['new_persons_one_dose'].cumsum()
    vacunation_df.loc[rows_current_province,'total_number_persons_two_doses'] = data_current_province['new_persons_two_doses'].cumsum()
    vacunation_df.loc[rows_current_province,'total_number_persons_three_doses'] = data_current_province['new_persons_three_doses'].cumsum()
    
    vacunation_df.loc[rows_current_province,'percentage_persons_one_dose'] = vacunation_df['total_number_persons_one_dose'] / population_province[current_province] * 100
    vacunation_df.loc[rows_current_province,'percentage_persons_two_doses'] = vacunation_df['total_number_persons_two_doses'] / population_province[current_province] * 100
    vacunation_df.loc[rows_current_province,'percentage_persons_three_doses'] = vacunation_df['total_number_persons_three_doses'] / population_province[current_province] * 100

### Final state of the dataset

In [18]:
vacunation_df.head(10)

,date,province,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses
0,2020-11-07,Segovia,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
1,2020-11-07,Soria,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
2,2020-11-07,Zamora,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
3,2020-11-07,TotalCyL,1,0,1,0.000042,0.000000,0.0,0.000000,1,1,0,0
4,2020-11-07,Burgos,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885,2022-04-12,Segovia,84,8,3,87.986421,85.831858,72.0,57.513127,333664,132231,128993,86434
4886,2022-04-12,Soria,46,3,0,91.481867,89.742707,44.0,63.271368,205408,80059,78537,55371
4887,2022-04-12,Burgos,267,140,23,90.564105,88.160623,119.0,58.877248,790092,315008,306648,204792
4888,2022-04-12,Palencia,78,7,3,91.430355,89.666624,66.0,62.359001,366297,143750,140977,98043


## Merging of both datasets

In [20]:
df = pd.merge(epidemiology_df, vacunation_df, how='outer', on=['date', 'province']).sort_values(by=['date', 'province']).reset_index(drop=True).fillna(0)
df.head(5)

,date,province,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses
0,2020-03-13,Burgos,90.0,29.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-13,León,32.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-13,Salamanca,11.0,5.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-13,Segovia,15.0,6.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
